# Data Cleaning

The purpose of this notebook is to clean unmodifed or modifed csv files collected from ECAAS ODK. 

The notebook contains the following steps:
1. Read in and display an ECAAS ODK csv
2. General Cleaning
3. Clean crop type attributes
5. Construct dictionary for renaming crops
6. Create geometries (working with ODK collected coordinates or working with manually edited coordinates)
9. Export data to geojson and Shapefile


## Load Packages

In [ ]:
import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon

pd.set_option("display.max_columns", None)

from create_geometries import (
    create_geometries_from_manual_edit,
    create_geometries_from_ODK,
)

## Read in ECAAS ODK csv

In [ ]:
samples_file = "data/Cleaned_ECAAS_ODK_Datasheet.csv"
samples_df = pd.read_csv(samples_file)

##  Initial inspection

To understand the kind of data available, it is useful to view the first 5 rows, and then view more detailed information

### First five rows of dataframe

In [ ]:
samples_df.head()

### Display column names, non-null count and dtype

In [ ]:
samples_df.info()

## General Cleaning

* Remove unnecessary prefixes from columns to improve readability
* Convert missing values from " " (single space) to None
* Convert start and end times from object to datetime

In [ ]:
# Remove prefixes from columns to improve readability
updated_columns = samples_df.columns.str.replace("data-", "")
updated_columns = updated_columns.str.replace("consent_given-", "")
updated_columns = updated_columns.str.replace("field_planted-", "")

samples_df.columns = updated_columns

In [ ]:
# Convert from missing values being " " to None
samples_df = samples_df.replace({" ": None})

In [ ]:
# Convert date columns to date strings. Must be string format for shapefile
samples_df["start"] = pd.to_datetime(samples_df["start"], dayfirst=True).dt.strftime("%Y-%m-%d")
samples_df["end"] = pd.to_datetime(samples_df["end"], dayfirst=True).dt.strftime("%Y-%m-%d")

### Display column names, non-null count and dtype after cleaning

In [ ]:
samples_df.info()

## Clean crop type attributes

The crop name of the primary crop is stored in the `primary_crop` field if selected from the toolkit list, and the `primary_crop_other` field otherwise. The same is true of the `secondary_crop` and `secondary_crop_other` columns. The following steps are used to produce single unified `primary_crop` and `secondary_crop` fields:

For each pair, the following steps are run:

1. Identify all rows containing a valid entry in the `crop_other` field.
2. Convert these entries to lower case to match entries in the `crop` field.
3. Copy valid entries from the `crop_other` field to the `crop` field.
4. Drop the `crop_other` field.

In [ ]:
prefixes = ["primary", "secondary"]

for prefix in prefixes:
    
    other_crop_condition = samples_df.loc[:, f"{prefix}_crop_other"].notna()
    
    samples_df.loc[other_crop_condition, f"{prefix}_crop"] = samples_df.loc[other_crop_condition, f"{prefix}_crop_other"].str.lower()
    
    samples_df = samples_df.drop(f"{prefix}_crop_other", axis="columns")

### View unique crops and value counts

Now that all crops have been collated into a single column and format, running `value_counts` will display the unique spellings and counts. After viewing this, you will need to compile a dictionary to coerce varied spellings for the same crop into a single spelling.

In [ ]:
# Get value counts for all instances in each crop column
for prefix in prefixes:
    
    print(samples_df[[f"{prefix}_crop"]].value_counts(
        dropna=False
    ).sort_index(ascending=True))

    print("---------")

### Construct dictionary for renaming crops

Based on the values in the list above, we compiled the following dictionary, then applied it to the `primary_crop` and `secondary_crop` fields. This step will rename appearances of the dictionary key to the corresponding value. For example "bananas" will become "banana".

After running the next two cells, review the cleaned crop list to check if any additions need to be made to the dictionary. Repeat as needed.

In [ ]:
# Create a dictionary to clean any remaining mismatched data
crop_dictionary = {
    "bananas": "banana",
    "groundnuts": "groundnut",
    "macadamia nuts": "macadamia",
    "macadamia nut": "macadamia",
    "ochra vegetables": "ochra",
    "okra": "ochra",
    "soyabean": "soyabean",
    "sweet potatoes": "sweet potato",
    "water melon": "watermelon",
}

In [ ]:
# Apply the dictionary to the crop columns
for prefix in prefixes:
    samples_df.loc[:, f"{prefix}_crop"] = samples_df.loc[:, f"{prefix}_crop"].replace(
        crop_dictionary
    )

    print(samples_df[f"{prefix}_crop"].value_counts())
    print("---------")

In [ ]:
import matplotlib.pyplot as plt

counts = samples_df['primary_crop'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

counts[counts >= 10].plot(kind='bar', ax=axes[0])
counts[counts < 10].plot(kind='bar', ax=axes[1])
axes[0].set_title("Crops with more than 10 samples")
axes[1].set_title("Crops with fewer than 10 samples")

## Create geometries

This section converts geometries collected by the ECAAS ODK toolkit from strings to Shapely geomerties for use with the Open Data Cube. It can process:
* A csv produced from the ECAAS ODK toolkit
* A modified version of the csv from the ECAAS ODK toolkit

---

### Working with ODK collected coordinates

The following logic is used to extract point and polygon geometries from the ODK output:

- If `access_consent == "yes"`, then row should have field boundary and field center
    - centre point is given as a "lat,lon" point
    - boundary is given as "lat lon alt acc; lat lon alt acc;" for each point collected along the boundary, separated by semi-colons
- If `access_consent == "no"`, then row should only have field outside corner (no boundary)
    - outer corner point is given as a "lat,lon"
    
For points, the method creates a new `point_location` location field, with values of either "center" or "outside corner".

---

### Working with manually edited coordinates

Specifically, after collecting points using the ECAAS ODK toolkit, an analyst may wish to manually update the points, for example, moving them from the road to within the field. If they have provided these as a new `lat,lon` pair in a new column of the csv, these points may be used instead of the original ODK points. 

Different cleaning routines will be run depending on the type of file supplied. The user may dictate which process to use by specifying the variable `cleaned_geom_column` as:
* `None` for an unmodified file (uses ODK collected coordinates)
* `"column_name"` for a modified file -- where `column_name` contains the modified lat,lon values in the csv (uses modified coordinates).

If using modified coordinates, the `point_location` field will be given as "manual"

In [ ]:
# use with original ODK toolkit output
# cleaned_geom_column = None

# use with modified ODK toolkit output. The column listed below must contain data in lat,lon format (i.e. -14.4,28.0)
cleaned_geom_column = "Cleaned_Coordinates"

In [ ]:
if cleaned_geom_column is None:
    # Return cleaned samples from ECAAS ODK format, containing mix of points and polygons
    cleaned_samples_df = create_geometries_from_ODK(samples_df)

    # Create two geodataframes, one with point geometry and one with polygon geometry
    points_gdf = cleaned_samples_df.drop(["field_boundary_polygon"], axis="columns").copy()
    polygons_gdf = cleaned_samples_df.set_geometry("field_boundary_polygon", drop=True).copy()
    polygons_gdf = polygons_gdf.loc[
        ~cleaned_samples_df["field_boundary_polygon"].isna(), :
    ]

else:
    # Retun geodataframe containing point geometry, extracted from cleaned_geom_column
    points_gdf = create_geometries_from_manual_edit(samples_df, cleaned_geom_column)
    polygons_gdf = None

## Export data to geojson and Shapefile

For the final output, we will export all points and polygons, as well as the cleaned single crop file. The cleaned single crop file will be used for the remainder of the machine learning training.

We define a dictionary for the column names to be shortened to when writing to the Shapefile format (10 character limit).

In [ ]:
# Create dictionary with columns of interest and corresponding 10-character names
col_rename_dict = {
    "start": "start",
    "end": "end",
    "field_fallow": "fallow",
    "primary_crop_type": "pri_type",
    "primary_crop": "pri_crop",
    "crop_development": "crop_dev",
    "multiple_crops": "multi_crop",
    "multiple_crops_percentage": "multi_per",
    "secondary_crop": "sec_crop",
    "geometry": "geometry",
}

# Create results directory if it doesn't exist
if not os.path.exists("results"):
    os.makedirs("results")
    
# Export polygons
if polygons_gdf is not None:

    polygons_gdf[col_rename_dict.keys()].to_file("results/cleaned_polygons.geojson")
    polygons_gdf[col_rename_dict.keys()].rename(columns=col_rename_dict).to_file(
        "results/cleaned_polygons.shp"
    )

# Add additional column to specify the point location
col_rename_dict["point_location"] = "point_loc"

# Export points
points_gdf[col_rename_dict.keys()].to_file("results/cleaned_points.geojson")
points_gdf[col_rename_dict.keys()].rename(columns=col_rename_dict).to_file(
    "results/cleaned_points.shp"
)